# Lesson 2: ETL/ELT Pipelines — Theory and Practice

**Module 3: Data & Pipeline Engineering**  
**Estimated Time**: 4-5 hours  
**Difficulty**: Intermediate

---

## 🎯 Learning Objectives

By the end of this lesson, you will:

✅ Understand the ETL (Extract, Transform, Load) pipeline architecture  
✅ Know when to choose ETL vs ELT and hybrid approaches  
✅ Build a complete multi-source hybrid ETL/ELT pipeline from scratch  
✅ Implement data validation, cleaning, and feature engineering in a pipeline  
✅ Answer 5 interview questions on pipeline design  

---

## 📚 Table of Contents

1. [ETL Foundations](#1-etl-foundations)
2. [The Extract Phase](#2-extract)
3. [The Transform Phase](#3-transform)
4. [The Load Phase](#4-load)
5. [ETL vs ELT — When to Use Which](#5-etl-vs-elt)
6. [Note on Streaming](#6-streaming)
7. [Hands-On: Building a Hybrid ETL/ELT Pipeline](#7-hands-on)
8. [Exercises](#8-exercises)
9. [Interview Preparation](#9-interview)

---

## 1. ETL Foundations <a id='1-etl-foundations'></a>

**ETL** stands for **Extract, Transform, Load**. It describes the pipeline of:
1. **Getting data** from sources
2. **Processing it** into a usable form
3. **Loading it** into a storage system for downstream use

ETL is often the **first stage** of preparing data for model training or inference.

### Why ETL Matters in MLOps

In traditional software engineering, the database schema is relatively fixed. In ML:
- Data **changes over time** (distribution drift)
- Features **evolve** as models improve
- Multiple **data sources** must be joined and reconciled
- **Reproducibility** requires deterministic pipelines
- **Data quality** directly impacts model performance

A well-designed ETL pipeline is the **backbone** of any production ML system.

---

## 2. The Extract Phase <a id='2-extract'></a>

Pull data from various sources (databases, APIs, files, logs, etc.).

### 2.1 Common Data Sources for ML

| Source | Protocol | Example |
|--------|----------|----------|
| Relational DB | SQL queries | PostgreSQL, MySQL |
| NoSQL DB | API/SDK | MongoDB, DynamoDB |
| Data Lake | File reads | S3 (Parquet), GCS |
| REST APIs | HTTP GET/POST | Third-party data |
| Message Queues | Consumer | Kafka, RabbitMQ |
| File Systems | File I/O | CSV, JSON uploads |

### 2.2 Validation During Extraction

**Critical principle:** Reject or quarantine bad data **as early as possible** to save trouble downstream.

```python
# Example: Early validation during extraction
def extract_with_validation(raw_records):
    valid = []
    quarantined = []
    for record in raw_records:
        if record.get('user_id') is None:
            quarantined.append(record)  # Missing required field
        elif not isinstance(record.get('amount'), (int, float)):
            quarantined.append(record)  # Invalid type
        else:
            valid.append(record)
    return valid, quarantined
```

**Best Practices:**
- Check for **malformed records** (missing fields, wrong types)
- Log or notify about quarantined data (don't silently drop!)
- Track **extraction metrics** (rows extracted, rows rejected, latency)

---

## 3. The Transform Phase <a id='3-transform'></a>

This is the **core processing step** — the "hefty" part where most data wrangling happens.

### 3.1 Common Transformations

| Type | Description | Example |
|------|-------------|----------|
| **Data Cleaning** | Handle issues in raw data | Fill missing values, remove duplicates |
| **Merging** | Join multiple data sources | Users + Orders + Products |
| **Standardization** | Consistent formats | Date formats, currency, casing |
| **Deduplication** | Remove duplicate records | Same order logged twice |
| **Aggregation** | Summarize data | Daily averages, counts |
| **Feature Engineering** | Create ML-ready features | `day-of-week`, `time-since-last-event` |
| **Encoding** | Convert to numeric | One-hot encoding, label encoding |
| **Normalization** | Scale numeric fields | StandardScaler, MinMaxScaler |

### 3.2 Feature Engineering in ETL

In ML terms, the Transform phase is where **raw data becomes features**:

| Raw Data | Engineered Feature |
|----------|-------------------|
| `timestamp: 2024-01-15 14:30:00` | `day_of_week: 1 (Monday)` |
| `last_purchase: 2023-12-20` | `days_since_last_purchase: 26` |
| `category: 'electronics'` | One-hot: `[1, 0, 0, 0]` |
| `price: 249.99` | `price_normalized: 0.72` |
| 5 transactions in 1 hour | `transaction_velocity: 5` |

### 3.3 Transform Best Practices

1. **Idempotency** — Running the transform twice should produce the same result
2. **Testability** — Each transform should be a testable function
3. **Documentation** — Document what each transform does and why
4. **Data validation** after transforms (not just before!)

---

## 4. The Load Phase <a id='4-load'></a>

Load the transformed data into a **target destination**.

### 4.1 Common Destinations in ML

| Destination | Use Case |
|-------------|----------|
| **Data Warehouse** (BigQuery, Redshift) | Analytical queries, dashboards |
| **Data Lake** (S3, GCS as Parquet) | ML training data storage |
| **Feature Store** (Feast) | Consistent training and serving features |
| **Database** (PostgreSQL) | Structured, queryable results |
| **File System** (local/cloud) | Training artifacts |

### 4.2 Load Strategies

| Strategy | Description | When to Use |
|----------|-------------|-------------|
| **Full Overwrite** | Replace all data | Small, easily reproduced datasets |
| **Append** | Add new records | Time-series, log data |
| **Upsert** | Insert or update | User profiles, slowly changing dimensions |
| **Partition Overwrite** | Replace one partition | Daily/hourly partitioned data |

### 4.3 Load Considerations

- **How often?** Batch (daily, hourly) vs streaming (real-time)
- **What format?** Parquet for analytics, databases for serving
- **Partitioning?** By date is most common (`year/month/day`)

**Example:** Load aggregated features daily into a warehouse table that the model training job reads:
```
s3://my-bucket/features/
  ├── year=2024/
  │   ├── month=01/
  │   │   ├── day=14/ features.parquet
  │   │   └── day=15/ features.parquet
```

---

## 5. ETL vs ELT — When to Use Which <a id='5-etl-vs-elt'></a>

### 5.1 ELT: Extract, Load, Transform

**ELT** is a variant where raw data is **first loaded** into storage (often a data lake) **before transformation**.

ELT has become popular with the rise of **inexpensive storage** and **scalable compute**. Organizations dump all raw data into a data lake immediately, and transform it later when needed.

### 5.2 Comparison

| Aspect | ETL | ELT |
|--------|-----|-----|
| **Transform timing** | Before loading | After loading |
| **Storage** | Only cleaned data stored | Raw + cleaned data stored |
| **Flexibility** | Less (schema decided upfront) | More (redefine transforms later) |
| **Ingestion speed** | Slower (processing first) | Faster (dump and process later) |
| **Risk** | Data loss (if you filter too aggressively) | Data swamp (too much unprocessed data) |
| **Cost** | Less storage, more compute upfront | More storage, compute on demand |
| **Best for** | Well-defined, stable pipelines | Exploratory, evolving ML pipelines |

### 5.3 The Hybrid Approach (Most Common in ML)

In practice, most ML teams use a **hybrid**:
1. **Light ETL** on extraction — basic validation, remove garbage
2. **Load** into a data lake (raw but validated)
3. **Heavy transforms** (feature engineering) later, closer to model training

### 5.4 Real-World Example: E-Commerce Recommendation

**ETL approach:**
An e-commerce company building a recommendation model:
- **Extract:** Pull data from production databases (orders, user info, product catalog)
- **Transform:** Join tables, clean inactive users, aggregate purchase history, encode categories
- **Load:** Save feature table to data warehouse

**ELT approach:**
- Dump all raw logs and databases into a data lake
- ML pipeline transforms raw data on the fly each time
- More flexible, but possibly slower

**Hybrid (recommended):**
- Some ETL to create intermediate validated features
- ELT of those into a warehouse
- Further transformations close to training time

---

## 6. Note on Streaming <a id='6-streaming'></a>

ETL traditionally implies **batch processing** (periodic loads). If you have **streaming data** (real-time) feeding into an online model, similar principles apply, but with streaming tools:

| ETL Stage | Batch Tool | Streaming Tool |
|-----------|-----------|----------------|
| Extract | SQL query, file read | **Kafka consumer** |
| Transform | Pandas, Spark (batch) | **Spark Streaming, Flink** |
| Load | Write to DB/file | **Write to Redis, Kafka topic** |

The foundational idea of **“get data → process → use data”** remains, just with low latency.

We’ll touch on streaming in the context of feature stores (Lesson 5) and orchestration (Lesson 7).

---

## 7. Hands-On: Building a Hybrid ETL/ELT Pipeline <a id='7-hands-on'></a>

In this section, we’ll simulate a basic ML data pipeline using Pandas, NumPy, and Scikit-learn.

### Objectives:
- Generate **synthetic data** to simulate data collection from multiple sources
- Explore different file formats
- Implement a **hybrid ETL/ELT approach** with practical data transformation workflows
- Obtain final processed DataFrames ready for ML

### Architecture Overview:
```
Source 1 (Users DB)  ────┐
                          ├─▶ EXTRACT ─▶ VALIDATE ─▶ LOAD (raw) ─▶ TRANSFORM ─▶ LOAD (features)
Source 2 (Events API) ───┘
```

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

np.random.seed(42)

# ============================================================
# STEP 1: SIMULATE DATA SOURCES
# Generating synthetic data to mimic real data sources
# ============================================================

print("="*60)
print("STEP 1: Simulating Data Sources")
print("="*60)

# --- Source 1: Users Database (Internal DB) ---
n_users = 10000
users_df = pd.DataFrame({
    'user_id': range(1, n_users + 1),
    'name': [f'User_{i}' for i in range(1, n_users + 1)],
    'email': [f'user_{i}@example.com' if np.random.random() > 0.05 
              else None for i in range(1, n_users + 1)],  # 5% missing emails
    'age': np.random.randint(18, 80, n_users),
    'country': np.random.choice(['US', 'UK', 'DE', 'FR', 'JP', 'IN', None], 
                               n_users, p=[0.3, 0.15, 0.1, 0.1, 0.1, 0.15, 0.1]),
    'signup_date': pd.date_range('2020-01-01', periods=n_users, freq='30min'),
    'subscription_tier': np.random.choice(['free', 'basic', 'premium'], 
                                          n_users, p=[0.6, 0.25, 0.15])
})

# Inject some messy data (realistic!)
users_df.loc[users_df.sample(50).index, 'age'] = -1  # Invalid ages
users_df.loc[users_df.sample(30).index, 'age'] = 150  # Unrealistic ages

print(f"Source 1 (Users DB): {users_df.shape[0]} records, {users_df.shape[1]} columns")
print(f"  Missing emails: {users_df['email'].isna().sum()}")
print(f"  Missing countries: {users_df['country'].isna().sum()}")
print(f"  Invalid ages: {((users_df['age'] < 0) | (users_df['age'] > 120)).sum()}")

# --- Source 2: Events/Transactions (API / Event Stream) ---
n_events = 50000
events_df = pd.DataFrame({
    'event_id': range(1, n_events + 1),
    'user_id': np.random.choice(range(1, n_users + 1), n_events),
    'event_type': np.random.choice(['purchase', 'view', 'add_to_cart', 'search', 'review'], 
                                   n_events, p=[0.1, 0.4, 0.2, 0.2, 0.1]),
    'product_category': np.random.choice(['electronics', 'clothing', 'food', 'books', 'sports'], 
                                         n_events),
    'amount': np.where(
        np.random.choice(['purchase', 'view', 'add_to_cart', 'search', 'review'], 
                         n_events, p=[0.1, 0.4, 0.2, 0.2, 0.1]) == 'purchase',
        np.round(np.random.exponential(50, n_events), 2),
        0.0
    ),
    'timestamp': pd.date_range('2024-01-01', periods=n_events, freq='1min')
})

# Inject duplicates (realistic in event streams!)
duplicates = events_df.sample(200)
events_df = pd.concat([events_df, duplicates], ignore_index=True)

print(f"\nSource 2 (Events API): {events_df.shape[0]} records, {events_df.shape[1]} columns")
print(f"  (includes {200} duplicate events)")

print("\n✅ Data sources simulated successfully!")

In [ ]:
# ============================================================
# STEP 2: EXTRACT WITH VALIDATION
# Apply early validation to reject/quarantine bad data
# ============================================================

print("="*60)
print("STEP 2: Extract with Validation")
print("="*60)

def validate_users(df):
    """Validate user records, quarantine invalid ones."""
    quarantine = []
    
    # Rule 1: age must be between 0 and 120
    invalid_age = (df['age'] < 0) | (df['age'] > 120)
    quarantine.append(('invalid_age', df[invalid_age]))
    
    # Rule 2: user_id must not be null
    invalid_id = df['user_id'].isna()
    quarantine.append(('missing_user_id', df[invalid_id]))
    
    # Remove quarantined records
    valid_mask = ~invalid_age & ~invalid_id
    valid_df = df[valid_mask].copy()
    
    return valid_df, quarantine

def validate_events(df):
    """Validate event records, remove duplicates."""
    # Remove exact duplicates
    before = len(df)
    df = df.drop_duplicates(subset=['event_id'])
    dupes_removed = before - len(df)
    
    # Validate: amount cannot be negative
    invalid_amount = df['amount'] < 0
    quarantine = [('negative_amount', df[invalid_amount])]
    df = df[~invalid_amount].copy()
    
    return df, quarantine, dupes_removed

# Apply validation
users_valid, users_quarantine = validate_users(users_df)
events_valid, events_quarantine, dupes = validate_events(events_df)

print(f"Users:  {len(users_df)} → {len(users_valid)} valid")
for reason, q_df in users_quarantine:
    if len(q_df) > 0:
        print(f"  Quarantined ({reason}): {len(q_df)} records")

print(f"\nEvents: {len(events_df)} → {len(events_valid)} valid")
print(f"  Duplicates removed: {dupes}")
for reason, q_df in events_quarantine:
    if len(q_df) > 0:
        print(f"  Quarantined ({reason}): {len(q_df)} records")

print("\n✅ Early validation complete! Bad data quarantined before proceeding.")

In [ ]:
# ============================================================
# STEP 3: LOAD RAW (ELT style - save validated raw data)
# This simulates loading into a data lake before heavy transforms
# ============================================================
import os

print("="*60)
print("STEP 3: Load Raw Data (ELT-style)")
print("="*60)

# Create a simulated "data lake" directory structure
os.makedirs('data_lake/raw/users', exist_ok=True)
os.makedirs('data_lake/raw/events', exist_ok=True)

# Save as Parquet (industry standard for data lakes)
users_valid.to_parquet('data_lake/raw/users/users_validated.parquet', index=False)
events_valid.to_parquet('data_lake/raw/events/events_validated.parquet', index=False)

# Check sizes
users_size = os.path.getsize('data_lake/raw/users/users_validated.parquet') / 1024
events_size = os.path.getsize('data_lake/raw/events/events_validated.parquet') / 1024

print(f"Saved to data lake:")
print(f"  data_lake/raw/users/  - {users_size:.0f} KB")
print(f"  data_lake/raw/events/ - {events_size:.0f} KB")
print("\n✅ Raw validated data loaded to data lake!")
print("\nThis is the ELT part: Load first, then Transform.")
print("Advantage: We can always go back to this raw data if we need different transforms.")

In [ ]:
# ============================================================
# STEP 4: TRANSFORM (Feature Engineering)
# This is the heavy-duty processing step
# ============================================================

print("="*60)
print("STEP 4: Transform (Feature Engineering)")
print("="*60)

# Read from the "data lake" (simulating a fresh pipeline run)
users = pd.read_parquet('data_lake/raw/users/users_validated.parquet')
events = pd.read_parquet('data_lake/raw/events/events_validated.parquet')

# --- Transform 1: Clean and standardize users ---
print("\nTransform 1: Cleaning users...")

# Fill missing countries with 'Unknown'
users['country'] = users['country'].fillna('Unknown')

# Fill missing emails with placeholder
users['email'] = users['email'].fillna('no_email@unknown.com')

# Create age bins (feature engineering)
users['age_group'] = pd.cut(users['age'], 
                            bins=[0, 25, 35, 50, 65, 120], 
                            labels=['18-25', '26-35', '36-50', '51-65', '65+'])

# Calculate account age
users['account_age_days'] = (pd.Timestamp('2024-06-01') - users['signup_date']).dt.days

print(f"  Added: age_group, account_age_days")
print(f"  Filled: {(users['country'] == 'Unknown').sum()} missing countries")

# --- Transform 2: Aggregate events per user ---
print("\nTransform 2: Aggregating events per user...")

user_event_features = events.groupby('user_id').agg(
    total_events=('event_id', 'count'),
    total_purchases=('event_type', lambda x: (x == 'purchase').sum()),
    total_views=('event_type', lambda x: (x == 'view').sum()),
    total_cart_adds=('event_type', lambda x: (x == 'add_to_cart').sum()),
    total_spend=('amount', 'sum'),
    avg_spend=('amount', 'mean'),
    num_categories=('product_category', 'nunique'),
    first_event=('timestamp', 'min'),
    last_event=('timestamp', 'max')
).reset_index()

# Derived features
user_event_features['purchase_rate'] = (
    user_event_features['total_purchases'] / 
    user_event_features['total_events']
).round(4)

user_event_features['cart_to_purchase_ratio'] = np.where(
    user_event_features['total_cart_adds'] > 0,
    user_event_features['total_purchases'] / user_event_features['total_cart_adds'],
    0
).round(4)

user_event_features['days_active'] = (
    (user_event_features['last_event'] - user_event_features['first_event']).dt.days
)

print(f"  Created {len(user_event_features.columns)} features per user")
print(f"  Users with events: {len(user_event_features)}")

# --- Transform 3: Join users + event features ---
print("\nTransform 3: Joining user profiles + event features...")

final_features = users.merge(user_event_features, on='user_id', how='left')

# Fill NaN for users with no events
event_cols = user_event_features.columns.drop('user_id')
for col in event_cols:
    if final_features[col].dtype in ['float64', 'int64']:
        final_features[col] = final_features[col].fillna(0)

print(f"  Final feature table: {final_features.shape}")
print(f"  Users with events: {(final_features['total_events'] > 0).sum()}")
print(f"  Users without events: {(final_features['total_events'] == 0).sum()}")

print("\n✅ Transformation complete!")
final_features.head()

In [ ]:
# ============================================================
# STEP 5: LOAD FEATURES (Final load for ML consumption)
# ============================================================

print("="*60)
print("STEP 5: Load Features (ML-ready)")
print("="*60)

os.makedirs('data_lake/features', exist_ok=True)

# Save the feature table
final_features.to_parquet('data_lake/features/user_features.parquet', index=False)
features_size = os.path.getsize('data_lake/features/user_features.parquet') / 1024

print(f"\nFeature table saved: {features_size:.0f} KB")
print(f"Shape: {final_features.shape}")
print(f"Columns: {list(final_features.columns)}")

# Pipeline summary
print("\n" + "="*60)
print("PIPELINE SUMMARY")
print("="*60)
print(f"Input:     {len(users_df):>6} users + {len(events_df):>6} events")
print(f"After validation: {len(users_valid):>6} users + {len(events_valid):>6} events")
print(f"Output:    {final_features.shape[0]:>6} rows x {final_features.shape[1]} columns")
print(f"\nPipeline stages: EXTRACT → VALIDATE → LOAD(raw) → TRANSFORM → LOAD(features)")
print("\n✅ Hybrid ETL/ELT pipeline complete!")

In [ ]:
# ============================================================
# STEP 6: POST-LOAD VALIDATION
# Always validate your output!
# ============================================================

print("="*60)
print("STEP 6: Post-Load Data Quality Checks")
print("="*60)

# Read back from storage to verify
loaded_features = pd.read_parquet('data_lake/features/user_features.parquet')

checks = [
    ('No null user_ids', loaded_features['user_id'].isna().sum() == 0),
    ('No negative ages', (loaded_features['age'] >= 0).all()),
    ('Total spend >= 0', (loaded_features['total_spend'] >= 0).all()),
    ('Purchase rate in [0,1]', loaded_features['purchase_rate'].between(0, 1).all()),
    ('Expected row count', len(loaded_features) == len(users_valid)),
    ('No duplicate user_ids', loaded_features['user_id'].nunique() == len(loaded_features)),
]

all_passed = True
for check_name, result in checks:
    status = '✅' if result else '❌'
    print(f"  {status} {check_name}")
    if not result:
        all_passed = False

print(f"\n{'All quality checks passed!' if all_passed else 'SOME CHECKS FAILED!'}")

# Cleanup simulated data lake
import shutil
shutil.rmtree('data_lake', ignore_errors=True)
print("\n(Cleaned up temporary files)")

## 8. Exercises <a id='8-exercises'></a>

### Exercise 1: Add a Third Data Source
Add a simulated "Product Catalog" data source with columns: `product_id, name, category, price, stock_count`. Integrate it into the pipeline by joining products with events.

### Exercise 2: Implement Idempotency
Modify the pipeline so that running it twice produces the same result. Hint: use timestamps or checksums to detect already-processed data.

### Exercise 3: Add Data Quality Metrics
Create a `pipeline_metrics` dictionary that tracks: total rows in, rows quarantined, duplicates removed, rows out, execution time per stage. Print a summary report.

---

## 9. Interview Preparation <a id='9-interview'></a>

### Q1: "Design an ETL pipeline for processing daily user activity data. How do you handle late-arriving data?"

**Answer:**  
"I’d design a pipeline with these stages:

1. **Extract**: Pull data from the activity log database daily at 2 AM UTC
2. **Transform**: Clean, deduplicate, compute daily aggregates per user
3. **Load**: Write to a partitioned Parquet table (partitioned by date)

For **late-arriving data**, I’d use a **lookback window** pattern:
- Each run processes data from the target day + a configurable lookback (e.g., 3 days)
- Use **partition overwrite** (not append) so reprocessing doesn’t create duplicates
- Track a `processing_timestamp` column separate from the event timestamp
- Flag rows that arrived late for monitoring

This makes the pipeline **idempotent** — running it multiple times produces the same result."

---

### Q2: "What’s the difference between ETL and ELT? When would you choose each?"

**Answer:**  
"In ETL, data is transformed before loading (clean → store). In ELT, raw data is loaded first, then transformed (store → clean).

I choose **ETL** when:
- Schema is well-defined and stable
- Storage is expensive (only store what you need)
- Compliance requires not storing raw PII

I choose **ELT** when:
- Exploration phase (don’t know what transforms needed yet)
- Storage is cheap (cloud data lakes)
- Multiple teams need different views of the same data

In practice, most ML teams use a **hybrid**: light validation during extraction, load raw to a data lake, then heavy feature engineering later. This preserves flexibility while maintaining data quality."

---

### Q3: "How do you ensure data quality in a production ETL pipeline?"

**Answer:**  
"I implement data quality at every stage:

1. **Extract**: Schema validation, null checks on required fields, type checking
2. **Transform**: Business rule validation (e.g., amount > 0), statistical checks (e.g., row count within expected range)
3. **Load**: Post-load verification (re-read and validate), row count comparison, checksum verification

Specific tools and patterns:
- **Great Expectations** for declarative data validation
- **Quarantine table** for invalid records (don’t silently drop!)
- **Data quality dashboards** with alerts for anomalies
- **Automated tests** that run before each pipeline stage proceeds"

---

### Q4: "How do you make a data pipeline idempotent? Why is this important?"

**Answer:**  
"An idempotent pipeline produces the same result regardless of how many times it runs.

Implementation strategies:
1. **Partition overwrite** instead of append (overwrite `date=2024-01-15` partition)
2. **Deduplication** using natural keys (event_id, user_id + timestamp)
3. **Deterministic transforms** — no `random()` without seeds, no `datetime.now()` in logic
4. **Upsert** operations for dimension tables

Why important:
- Failed jobs can be safely **retried** without data corruption
- **Backfill** operations (reprocessing historical data) just work
- Debugging is easier when you can reproduce results"

---

### Q5: "How would you debug a failed ETL job in production?"

**Answer:**  
"My debugging workflow:

1. **Check the logs** — identify which stage failed (extract, transform, or load)
2. **Check input data** — has the source schema changed? Any corrupt records?
3. **Check environment** — disk space, memory, network connectivity, credentials
4. **Reproduce locally** — pull a sample of the failing data and run the pipeline locally
5. **Root cause analysis** — is it a data issue, code bug, or infrastructure issue?
6. **Fix and test** — add a test case for this exact failure mode
7. **Backfill** — reprocess the failed data (idempotent pipeline makes this safe)

Prevention:
- **Structured logging** with correlation IDs
- **Alerting** on pipeline failures with context (not just ‘pipeline failed’)
- **Dead letter queues** for records that fail transformation"

---

## 🎓 Key Takeaways

1. **ETL is Extract → Transform → Load** — the backbone of ML data pipelines
2. **Validate early** — reject bad data at extraction, not after expensive transforms
3. **ETL vs ELT** — most ML teams use a hybrid approach
4. **Idempotency** is non-negotiable for production pipelines
5. **Data quality checks** should exist at every stage, not just at the end
6. The **Transform phase** is where raw data becomes ML features

---

➡️ **Next Lesson**: [Lesson 3: Sampling Strategies](./lesson_03_sampling_strategies.ipynb) — Learn how sampling choices affect your models.